# Creating an end to end fastai2 model for the dataset

inspiration from : 
1. https://www.kaggle.com/nikkisharma536/movie-review-sentiment-deep-learning-fastai - much of the code is depreciated.`
2. this was a massive help - https://www.kaggle.com/muellerzr/end-to-end-fastai2


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip
/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv
/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip


In [8]:
from fastai.text.all import *

In [9]:
path = '/kaggle/input/sentiment-analysis-on-movie-reviews/'

In [10]:
from zipfile import ZipFile

with ZipFile(path+'train.tsv.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working')
    
with ZipFile(path+'test.tsv.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working')
    


In [11]:
!dir '../output/kaggle/working'

test.tsv  train.tsv


In [12]:
path = '../output/kaggle/working'

In [13]:
train_df = pd.read_csv(path + '/train.tsv', sep='\t')
test_df = pd.read_csv(path + '/test.tsv', sep='\t')
train_df['Sentiment'] = train_df['Sentiment'].apply(str)

In [14]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [15]:
test_df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [16]:
test_id = test_df['PhraseId']

In [17]:
test_df['PhraseId']

0        156061
1        156062
2        156063
3        156064
4        156065
          ...  
66287    222348
66288    222349
66289    222350
66290    222351
66291    222352
Name: PhraseId, Length: 66292, dtype: int64

In [18]:
test_df['Phrase'][0]

'An intermittently pleasing but mostly routine effort .'

In [19]:
train_df['Sentiment'].unique()

array(['1', '2', '3', '4', '0'], dtype=object)

In [20]:
# data = (TextList.from_df(train,cols='Phrase').split_by_rand_pct(0.2).label_for_lm().databunch(bs=48))
# data.show_batch()

blocks = (TextBlock.from_df(text_cols='Phrase', is_lm=True, res_col_name='text'))


In [21]:
## following along to meullers code here
get_x = ColReader('text')
splitter = RandomSplitter(0.1, seed=42)

In [22]:
data_block = DataBlock(blocks=blocks,get_x=get_x,splitter=splitter)

In [23]:
type(test_df)

pandas.core.frame.DataFrame

In [24]:
text_df = pd.Series.append(train_df['Phrase'], test_df['Phrase'])

In [25]:
text_df = pd.DataFrame(text_df)

In [26]:
text_df.head()

,Phrase
0,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story ."
1,A series of escapades demonstrating the adage that what is good for the goose
2,A series
3,A
4,series


In [27]:
data = data_block.dataloaders(text_df, bs=64)

In [28]:
learn = language_model_learner(data,AWD_LSTM, pretrained=True,metrics=[accuracy, Perplexity()])

In [29]:
learn.to_fp16()
learn.fine_tune(1, 4e-3)
# learn.fine_tune(10,4e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.103606,4.488070,0.284866,88.949570,04:01


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.547584,4.112233,0.321599,61.082973,04:11


### Classification

In [35]:
#ys = ['1', '2','3','4', '0'] # this is wrong
ys = ['Sentiment']

In [36]:
blocks = (TextBlock.from_df('Phrase', seq_len=data.seq_len, vocab=data.vocab),
         MultiCategoryBlock(encoded=True, vocab=ys))

In [37]:
class_dblock = DataBlock(blocks=blocks,
                        get_x=ColReader('text'),
                        get_y=ColReader(ys),
                        splitter=RandomSplitter())

In [38]:
#class_dblock.summary(train_df.iloc[:100])

Setting-up type transforms pipelines
0          1           1   
1          2           1   
2          3           1   
3          4           1   
4          5           1   
..       ...         ...   
95        96           3   
96        97           3   
97        98           3   
98        99           3   
99       100           3   

                                                                                                                                                                                          Phrase  \
0   A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .   
1                                                                                                                  A series of escapades demonstrating the adage that what is good for the goose   
2                                                                 

Setting up Pipeline: ColReader -- {'cols': ['Sentiment'], 'pref': '', 'suff': '', 'label_delim': None} -> EncodedMultiCategorize -- {'vocab': ['Sentiment'], 'sort': False, 'add_na': False}


TypeError: new(): invalid data type 'str'

In [39]:
dls = class_dblock.dataloaders(train_df)

TypeError: new(): invalid data type 'str'

In [40]:
loss_func = BCEWithLogitsLossFlat(thresh=0.8)
metrics = [partial(accuracy_multi, thresh=0.8)]

In [ ]:
class_learn = text_classifier_learner(dls, AWD_LSTM, metrics=metrics, loss_func=loss_func)

In [ ]:
learn.lr_find()

In [ ]:
learn.load_encoder('fine_tuned')

### ulfit methodology for lr find

In [ ]:
# learn.to_fp16()
# lr = 1e-2
# moms = (0.8,0.7, 0.8)
# lr *= learn.dls.bs/128
# learn.fit_one_cycle(1, lr, moms=moms, wd=0.1)

In [ ]:
# learn.freeze_to(-2)
# lr/=2
# learn.fit_one_cycle(1, slice(lr/(2.6**4), lr), moms=moms, wd=0.1)

In [ ]:
# learn.freeze_to(-3)
# lr /=2
# learn.fit_one_cycle(1, slice(lr/(2.6**4), lr), moms=moms, wd=0.1)

In [ ]:
# learn.unfreeze()
# lr /= 5
# learn.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7,0.8), wd=0.1)

### submitting the prediction

In [ ]:
dl = learn.dls.test_dl(test_df['Phrase'])

In [ ]:
preds = learn.get_preds(dl=dl)

In [ ]:
sub = pd.read_csv(path+'/sample_submission.csv')

In [ ]:
sub.head()

In [ ]:
preds[0][0].cpu().numpy()

In [ ]:
sub[ys] = preds[0]

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)